# 0. Configuración de librerias y requisitos previos

In [185]:
"""
Se importan las librerías necesarias para el procesamiento de texto y manejo de archivos.

- nltk.tokenize.regexp_tokenize: para tokenización basada en expresiones regulares.
- nltk.stem.SnowballStemmer: para aplicar stemming en inglés.
- gensim: para vectorización de los documentos y calculo de similaridad. 
- xml.etree.ElementTree: para parseo de archivos en formato XML.
- os: para operaciones relacionadas con el sistema de archivos.
- typing: para anotaciones de tipo.
- numpy: manejo del tipado de las métricas.
"""

from nltk.tokenize import regexp_tokenize
from nltk.stem import SnowballStemmer

from gensim import corpora, models, similarities

import xml.etree.ElementTree as ET

from typing import List, Dict, Union, Set, Tuple

import numpy as np

import os

In [186]:
"""
Carga de stopwords desde un archivo local.

- Evita la descarga de NLTK y permite usar una lista personalizada de palabras vacías.
- Se construye el conjunto `stop_words` para filtrar palabras vacías durante el preprocesamiento.
"""

# NO cambie esta ruta
path_stopwords = "../data/stopwords/english"

def load_stopwords(
        file_path: str
    ) -> Set[str]:
    
    """
    Carga stopwords desde un archivo de texto plano.

    Cada línea del archivo debe contener una palabra. La función convierte
    todas las palabras a minúsculas y elimina líneas vacías.

    Parámetros
    ----------
    file_path : str
        Ruta del archivo de stopwords.

    Retorna
    -------
    Set[str]
        Conjunto de palabras que deben considerarse stopwords.
    """
    
    with open(file_path, "r", encoding = "utf-8") as file:
        stop_words = {line.strip().lower() for line in file if line.strip()}
    return stop_words

stop_words = load_stopwords(path_stopwords)

In [187]:
"""
Configuración del stemmer y definición del patrón de tokenización.

- Se inicializa `SnowballStemmer("english")` para reducir las palabras a su raíz 
  durante el preprocesamiento.
- Se define la expresión regular `pattern` que controla cómo se segmenta el texto
  en tokens. Incluye reglas para:
    * Abreviaturas (e.g., U.S.A.)
    * Palabras con guiones internos (e.g., non-euclidean)
    * Monedas, números y porcentajes
    * Puntos suspensivos (...)
    * Signos de puntuación como tokens separados
"""

stemmer = SnowballStemmer("english")

pattern = r'''(?x)              
    (?:[A-Z]\.)+[A-Z]?          # abreviaturas, e.g. U.S.A.
  | [A-Za-z]+(?:-[A-Za-z]+)*    # palabras con guiones internos, e.g. non-euclidean
  | \$?\d+(?:\.\d+)?%?          # monedas, números, porcentajes
  | \.\.\.                      # puntos suspensivos
  | [][.,;"'?():_`-]            # puntuación como tokens separados
'''

# 1. Preprocesamiento del texto

In [188]:
def preprocess(
		text: str
	) -> List[str]:
    
	"""
    Preprocesa un texto aplicando tokenización, normalización, eliminación de stopwords y stemming.

    El proceso de preprocesamiento transforma un texto crudo en una lista de tokens
    limpios y estandarizados, listos para tareas de recuperación de información o NLP.

    Pasos:
        1. Tokenización: se divide el texto en palabras usando la expresión regular `pattern`.
        2. Normalización: se convierten todos los tokens a minúsculas.
        3. Eliminación de stopwords: se filtran palabras vacías presentes en `stop_words`.
        4. Stemming: se aplica `SnowballStemmer` para reducir los tokens a su raíz.

    Parámetros
    ----------
    text : str
        Texto de entrada a procesar.

    Retorna
    -------
    List[str]
        Lista de tokens preprocesados.
    """
      
	# Generación de tokens
	tokens = regexp_tokenize(text, pattern)
	# Normalización 
	tokens = [token.lower() for token in tokens]
	# Eliminación de palabras de parada
	tokens = [token for token in tokens if token not in stop_words]
	# Stemming 
	tokens = [stemmer.stem(token) for token in tokens]
	return tokens

In [189]:
"""
Ejemplo de ejecución del preprocesamiento de texto.

Se procesa un texto de ejemplo que incluye números, signos de puntuación y abreviaturas. 
"""

example_text = "Se deben invertir $12.000 millones en D.I.A ..."

print("--------** preprocess **--------\n")
print("Parametros:\n")
print("- Texto de entrada = ", example_text)
print("\nResultado:\n")
print("- Tokens procesados = ", preprocess(example_text))


--------** preprocess **--------

Parametros:

- Texto de entrada =  Se deben invertir $12.000 millones en D.I.A ...

Resultado:

- Tokens procesados =  ['se', 'deben', 'invertir', '$12.000', 'millon', 'en', 'd.i.a', '...']


# 2. Carga de documentos

In [190]:
"""
Ruta de los documentos a procesar.

- `path_docs` indica la carpeta donde se encuentran los archivos de texto crudos.
- Cambie esta ruta según la ubicación de sus documentos en su sistema.
"""

# CAMBIE esta ruta según la ubicación de los archivos en su sistema
path_docs = "../data/docs-raw-texts"

In [191]:
def load_documents(
		path_docs: str
	) -> Dict[str, List[str]]:

	"""
    Carga y preprocesa los documentos de una carpeta específica.

    La función recorre todos los archivos con extensión `.naf` en la ruta
    especificada, extrae el ID del documento, el título y el contenido crudo,
    concatena título y contenido, y aplica la función `preprocess` para 
    obtener los tokens preprocesados de cada documento.

    Parámetros
    ----------
    path_docs : str
        Ruta de la carpeta que contiene los archivos `.naf` a procesar.
        Cambie esta ruta según la ubicación de sus documentos.

    Retorna
    -------
    Dict[str, List[str]]
        Diccionario donde las claves son los `doc_id` de cada documento y
        los valores son listas de tokens preprocesados.
    """
	
	docs = {}
	for file_name in os.listdir(path_docs):
		# Solo se revisan los archivos con extensión correcta
		if not file_name.endswith(".naf"):
			continue
		tree = ET.parse(os.path.join(path_docs, file_name))
		root = tree.getroot()
		# Id recuperado desde el atributo pupblicId
		doc_id = root.find("nafHeader/public").attrib["publicId"]
		# Titulo desde el atributo title
		title = root.find("nafHeader/fileDesc").attrib.get("title", "")
		# Contenido desde <raw>
		raw_element = root.find("raw")
		content = raw_element.text if raw_element is not None else ""
		# concatenar titulo + contenido para preprocesar
		tokens = preprocess(title +  " " + content)
		docs[doc_id] = tokens
	return docs

In [192]:
"""
Ejemplo de carga y preprocesamiento de documentos.

Se utiliza la función `load_documents` para cargar todos los archivos `.naf`
de la carpeta especificada en `path_docs`. Cada documento se preprocesa y se almacena
en un diccionario donde la clave es el `doc_id` y el valor es la lista de tokens.

Luego se muestra el contenido preprocesado de un documento específico
con ID "d006".
"""

documents = load_documents(path_docs)
example_id = "d006"

print("--------** load documents **--------\n")
print("Parametros:\n")
print("- Ruta de los documentos = ", path_docs)
print("\nResultado:\n")
print("- Documentos cargados = ", len(documents))
print("- Ejemplo documento " + example_id + " = ", documents[example_id])

--------** load documents **--------

Parametros:

- Ruta de los documentos =  ../data/docs-raw-texts

Resultado:

- Documentos cargados =  331
- Ejemplo documento d006 =  ['eugenio', 'beltrami', 'non-euclidian', 'geometri', 'eugenio', 'beltrami', 'non-euclidian', 'geometri', '.', 'eugenio', 'beltrami', '(', '1835', '-', '1900', ')', '.', 'novemb', '16', ',', '1835', ',', 'italian', 'mathematician', 'eugenio', 'beltrami', 'born', '.', 'notabl', 'work', 'concern', 'differenti', 'geometri', 'mathemat', 'physic', '.', 'work', 'note', 'especi', 'clariti', 'exposit', '.', 'first', 'prove', 'consist', 'non-euclidean', 'geometri', 'model', 'surfac', 'constant', 'curvatur', ',', 'pseudospher', '.', 'eugenio', 'beltrami', 'born', 'cremona', 'lombardi', ',', 'part', 'austrian', 'empir', ',', 'part', 'itali', '.', 'son', 'artist', 'paint', 'miniatur', ',', 'young', 'eugenio', 'certain', 'inherit', 'artist', 'talent', 'famili', ',', 'case', 'addit', 'mathemat', 'talent', 'would', 'acquir', ',', 'm

# 3. Representación vectorial Tf-IDf

In [193]:
def build_tfidf_model(
            tokenized_docs: List[List[str]]
    ) -> Tuple[corpora.Dictionary, List[List[Tuple[int, int]]], models.TfidfModel]:
    
	"""
    Construye un modelo TF-IDF a partir de un conjunto de documentos tokenizados usando Gensim.

    Parámetros
    ----------
    tokenized_docs : List[List[str]]
        Lista de documentos, donde cada documento es una lista de tokens (palabras ya preprocesadas).

    Retorna
    -------
    dictionary : corpora.Dictionary
        Diccionario que asigna un ID único a cada token presente en la colección.
    corpus : List[List[Tuple[int, int]]]
        Representación BoW (bag-of-words) de los documentos, donde cada documento
        es una lista de tuplas (id_token, frecuencia).
    tfidf_model : models.TfidfModel
        Modelo TF-IDF entrenado sobre el corpus, que permite transformar documentos
        de la representación BoW a vectores ponderados TF-IDF.
    """
      
	dictionary = corpora.Dictionary(tokenized_docs)
	corpus = [dictionary.doc2bow(doc) for doc in tokenized_docs]
	tfidf_model = models.TfidfModel(corpus)
	return dictionary, corpus, tfidf_model


In [194]:
"""
Ejemplo de construcción de un modelo TF-IDF.

Se utiliza la función `build_tfidf_model` para:
1. Recibir los documentos tokenizados en `tokenized_docs`.
2. Crear un diccionario de términos con identificadores únicos.
3. Representar los documentos en formato Bag-of-Words (`corpus`).
4. Entrenar un modelo TF-IDF sobre el corpus.

Luego se muestra:
- El diccionario con los tokens registrados.
- La representación BoW de un documento de ejemplo.
- La representación TF-IDF del mismo documento.
"""

tokenized_docs = list(documents.values())

dictionary, corpus, tfidf_model = build_tfidf_model(tokenized_docs)

print("--------** build tfidf model **--------\n")
print("Parámetros:\n")
print("- Número de documentos = ", len(tokenized_docs))
print("\nResultado:\n")
print("- Diccionario = ", dictionary)
print("- Documento ejemplo (BoW) = ", corpus)
print("- Documento ejemplo (TF-IDF) = ", tfidf_model)


--------** build tfidf model **--------

Parámetros:

- Número de documentos =  331

Resultado:

- Diccionario =  Dictionary<14103 unique tokens: ['(', ')', ',', '-', '.']...>
- Documento ejemplo (BoW) =  [[(0, 4), (1, 4), (2, 48), (3, 2), (4, 34), (5, 3), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 3), (15, 1), (16, 2), (17, 1), (18, 1), (19, 1), (20, 1), (21, 2), (22, 1), (23, 1), (24, 1), (25, 2), (26, 1), (27, 1), (28, 1), (29, 10), (30, 1), (31, 1), (32, 4), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 3), (49, 2), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 2), (57, 1), (58, 1), (59, 3), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 2), (67, 1), (68, 1), (69, 1), (70, 3), (71, 1), (72, 1), (73, 1), (74, 2), (75, 1), (76, 1), (77, 2), (78, 1), (79, 3), (80, 1), (81, 16), (82, 1), (83, 2), (84, 2), (85, 1), (86, 1), (87, 2), (88, 1)

In [195]:
def build_similarity_index(
        tfidf_corpus: List[List[tuple[int, float]]], 
        dictionary: corpora.Dictionary, 
        tfidf_model: models.TfidfModel
    ) -> similarities.SparseMatrixSimilarity:
    
    """
    Construye un índice de similitud basado en representaciones TF-IDF.

    Esta función utiliza el corpus ya transformado en TF-IDF junto con el diccionario
    y el modelo TF-IDF entrenado para generar una estructura eficiente que permite 
    calcular similitudes entre un documento de consulta y todos los documentos 
    de la colección.

    Args:
        tfidf_corpus (List[List[tuple[int, float]]]): 
            El corpus de documentos representado en formato BoW y transformado a TF-IDF.
        dictionary (Dictionary): 
            Diccionario de términos creado a partir de los documentos tokenizados.
        tfidf_model (TfidfModel): 
            Modelo TF-IDF entrenado sobre el corpus.

    Returns:
        similarities.SparseMatrixSimilarity: 
            Un índice de similitud esparso que permite realizar búsquedas eficientes 
            basadas en la similitud de coseno entre documentos en el espacio TF-IDF.
    """
    
    return similarities.SparseMatrixSimilarity(
        tfidf_model[tfidf_corpus],
        num_features = len(dictionary)
    )

In [196]:
"""
Ejemplo de construcción del índice de similitud TF-IDF.

Se utiliza la función `build_similarity_index` para crear un índice de similitud 
basado en los documentos representados en el espacio TF-IDF. Este índice permite 
realizar comparaciones rápidas entre documentos y consultas.

En este ejemplo, se construye el índice a partir del `dictionary`, `corpus` y 
`tfidf_model` previamente generados. Finalmente, se imprime información del 
índice creado.
"""

similarity_index = build_similarity_index(corpus, dictionary, tfidf_model)

print("--------** build similarity index **--------\n")
print("Parametros:\n")
print("- Número de documentos en el corpus =", len(corpus))
print("- Número de términos en el diccionario =", len(dictionary))
print("\nResultado:\n")
print("- Tipo de índice =", type(similarity_index))
print("- Índice Similaridad =", similarity_index)


--------** build similarity index **--------

Parametros:

- Número de documentos en el corpus = 331
- Número de términos en el diccionario = 14103

Resultado:

- Tipo de índice = <class 'gensim.similarities.docsim.SparseMatrixSimilarity'>
- Índice Similaridad = <gensim.similarities.docsim.SparseMatrixSimilarity object at 0x7caffdc99210>


In [197]:
def retrieve_documents(
        query_tokens: List[str], 
        dictionary: corpora.Dictionary, 
        tfidf_model: models.TfidfModel, 
        index: similarities.SparseMatrixSimilarity, 
        all_doc_ids: List[str]
    ) -> List[Tuple[str, float]]:
    
	"""
    Recupera documentos relevantes para una consulta en base a similitud TF-IDF.

    Esta función convierte una consulta tokenizada en su representación BoW, 
    la transforma al espacio TF-IDF y calcula la similitud de coseno entre la consulta 
    y todos los documentos de la colección. 

    Posteriormente, ordena los documentos de mayor a menor similitud y 
    devuelve únicamente aquellos con puntaje mayor que 0.

    Args:
        query_tokens (List[str]): 
            Lista de tokens que representan la consulta.
        dictionary (Dictionary): 
            Diccionario de términos creado a partir de los documentos tokenizados.
        tfidf_model (TfidfModel): 
            Modelo TF-IDF entrenado sobre el corpus.
        index (SparseMatrixSimilarity): 
            Índice de similitud creado a partir del corpus TF-IDF.
        all_doc_ids (List[str]): 
            Lista con los IDs de todos los documentos en el corpus.

    Returns:
        List[Tuple[str, float]]: 
            Lista de tuplas (doc_id, score) ordenadas por relevancia, 
            donde `score` es el puntaje de similitud con la consulta.
    """
    
	query_bow = dictionary.doc2bow(query_tokens)
	query_tfidf = tfidf_model[query_bow]
	# Arreglo con los valores de similitud
	sims = index[query_tfidf]
	ranked = sorted(zip(all_doc_ids, sims), key = lambda x: x[1], reverse = True)

	# Filtramos solo puntajes mayores 0
	return [(doc, float(score)) for doc, score in ranked if score > 0]


In [198]:
"""
Ejemplo de recuperación de documentos con consulta en TF-IDF.

Se utiliza la función `retrieve_documents` para obtener los documentos más 
relevantes en relación con una consulta tokenizada. La consulta se convierte 
al espacio TF-IDF y se compara contra todos los documentos de la colección 
usando el índice de similitud.

En este ejemplo, la consulta contiene los términos ["hospital", "paciente"]. 
Se recuperan los documentos ordenados por puntaje de similitud.
"""

query_tokens = ["person", "hannov"]
retrieved = retrieve_documents(query_tokens, dictionary, tfidf_model, similarity_index, list(documents.keys()))

print("--------** retrieve documents **--------\n")
print("Parametros:\n")
print("- Consulta tokens =", query_tokens)
print("\nResultado:\n")
print("- Documentos recuperados =", len(retrieved))
print("- Top 5 documentos:")
for doc_id, score in retrieved[:5]:
    print(f"  {doc_id}: {score:.4f}")


--------** retrieve documents **--------

Parametros:

- Consulta tokens = ['person', 'hannov']

Resultado:

- Documentos recuperados = 70
- Top 5 documentos:
  d150: 0.0617
  d268: 0.0547
  d174: 0.0513
  d291: 0.0383
  d128: 0.0342


In [199]:
def process_queries_folder_gensim(
        path_queries: str,
        dictionary: corpora.Dictionary,
        tfidf_model: models.TfidfModel,
        similarity_index: similarities.SparseMatrixSimilarity,
        all_doc_ids: List[str],
        output_file: str
    ) -> None:
    
	"""
    Procesa un conjunto de queries en formato `.naf` y recupera documentos usando
    TF-IDF y un índice de similitud de Gensim.

    Cada archivo `.naf` contiene una consulta que se preprocesa y se transforma
    en un vector TF-IDF. Posteriormente, se calcula la similitud con todos los 
    documentos del corpus a través del índice, se ordenan los resultados y se 
    escriben en un archivo de salida.

    Args:
        path_queries (str):
            Ruta de la carpeta que contiene las queries en formato `.naf`.
        dictionary (corpora.Dictionary):
            Diccionario entrenado a partir de los documentos tokenizados.
        tfidf_model (models.TfidfModel):
            Modelo TF-IDF entrenado sobre el corpus.
        similarity_index (similarities.SparseMatrixSimilarity):
            Índice de similitud basado en TF-IDF para el corpus.
        all_doc_ids (List[str]):
            Lista de identificadores de documentos en el mismo orden que el índice.
        output_file (str):
            Ruta del archivo donde se escribirán los resultados de las consultas.

    Returns:
        None
            La función no retorna ningún valor, pero escribe en `output_file` los
            resultados de cada consulta en el siguiente formato:

            ```
            query_id
            doc1:score1,doc2:score2,doc3:score3
            ```
            donde los documentos se listan en orden descendente de similitud.
    """
    
	with open(output_file, "w", encoding = "utf-8") as out_file:
		# Recorrido ordenado por id de query
		for fname in sorted(os.listdir(path_queries)):
			# Solo se procesan archivos válidos
			if fname.endswith(".naf"):
				file_path = os.path.join(path_queries, fname)
				tree = ET.parse(file_path)
				root = tree.getroot()
				# Extracción del id de la query
				public_elem = root.find(".//public")
				query_id = public_elem.attrib.get("publicId")
				# Texto de consulta
				raw_elem = root.find("raw")
				content = raw_elem.text if raw_elem is not None else ""
				query_tokens = preprocess(content)
				# Vector TF-IDF de la consulta
				bow = dictionary.doc2bow(query_tokens)
				query_vec = tfidf_model[bow]
				# Puntajes de similitud
				sims = similarity_index[query_vec]
				# Asociar cada doc_id con su score (solo scores > 0)
				scores = {
					doc_id: float(score)
					for doc_id, score in zip(all_doc_ids, sims) if score > 0
				}
				# Ordenar documentos por score
				ranked = sorted(scores.items(), key=lambda x: x[1], reverse=True)
				out_file.write(f"{query_id}\n")
				if ranked:
					line = ",".join([f"{doc}:{score:.4f}" for doc, score in ranked])
					out_file.write(line + "\n")

In [200]:
"""
Ejemplo de procesamiento y ranking de consultas con Gensim.

Se utiliza la función `process_queries_folder_gensim` para:
1. Leer todas las consultas en formato `.naf` desde `path_queries`. Puede cambiar la ruta.
2. Preprocesar cada consulta y construir su vector TF-IDF.
3. Calcular la similitud coseno entre la consulta y los documentos del corpus,
   usando el `dictionary`, el `tfidf_model` y el `index` de Gensim.
4. Guardar el ranking de documentos relevantes en `output_file`.

Luego se muestra el contenido del archivo de salida para verificar los resultados.
"""

# CAMBIE estas rutas según la ubicación en su sistema
path_queries = "../data/queries-raw-texts"
output_file = "../results/GESIM-consultas_resultados.tsv"

process_queries_folder_gensim(
    path_queries,
    dictionary,
    tfidf_model,
    similarity_index,
    list(documents.keys()),
    output_file
)

print("--------** process queries gensim **--------\n")
print("Parámetros:\n")
print("- Carpeta de consultas =", path_queries)
print("- Archivo de resultados =", output_file)
print("\nResultado (primeras líneas):\n")

# Mostrar las primeras 5 líneas del archivo de salida
with open(output_file, "r", encoding="utf-8") as f:
    for i, line in enumerate(f):
        print(line.strip())
        if i >= 4:
            break


--------** process queries gensim **--------

Parámetros:

- Carpeta de consultas = ../data/queries-raw-texts
- Archivo de resultados = ../results/GESIM-consultas_resultados.tsv

Resultado (primeras líneas):

q01
d016:0.1116,d085:0.0677,d259:0.0654,d254:0.0642,d186:0.0475,d209:0.0454,d215:0.0357,d008:0.0326,d153:0.0319,d170:0.0295,d163:0.0278,d185:0.0266,d154:0.0238,d315:0.0228,d089:0.0215,d296:0.0214,d060:0.0193,d004:0.0178,d006:0.0175,d099:0.0173,d162:0.0163,d243:0.0154,d100:0.0152,d094:0.0143,d179:0.0143,d039:0.0139,d145:0.0136,d059:0.0128,d312:0.0120,d311:0.0113,d329:0.0106,d299:0.0104,d065:0.0103,d074:0.0102,d130:0.0101,d172:0.0100,d028:0.0098,d082:0.0098,d273:0.0095,d281:0.0090,d255:0.0085,d152:0.0085,d077:0.0085,d317:0.0084,d195:0.0079,d212:0.0078,d284:0.0075,d275:0.0075,d265:0.0075,d164:0.0074,d052:0.0074,d032:0.0073,d316:0.0069,d229:0.0068,d136:0.0068,d024:0.0063,d021:0.0063,d234:0.0061,d038:0.0061,d116:0.0060,d123:0.0058,d184:0.0057
q02
d147:0.1198,d149:0.0809,d283:0.0731,d29

# 4. Evaluación de los resultados 

In [201]:
def load_relevance_judgments(
    	path: str
	) -> Dict[str, Dict[str, int]]:
    
	"""
    Carga los juicios de relevancia desde un archivo.

    La función lee un archivo de texto donde cada línea contiene un identificador
    de consulta (`query_id`) y un conjunto de pares `doc_id:score` separados por
    comas, que indican la relevancia de cada documento respecto a la consulta.
    Los valores de relevancia suelen expresarse como enteros (por ejemplo, 0 = no
    relevante, 1 = relevante).

    El archivo debe tener el siguiente formato:

        query1    doc1:1,doc2:0,doc3:1
        query2    doc2:1,doc4:1
        ...

    Parámetros
    ----------
    path : str
        Ruta del archivo de juicios de relevancia codificados en formato TSV.
        Cada línea debe contener: `query_id <tab> doc_id:score,doc_id:score,...`

    Retorna
    -------
    Dict[str, Dict[str, int]]
        Diccionario donde las claves son los `query_id` y los valores son
        diccionarios con los `doc_id` como claves y las puntuaciones de relevancia
        como valores. Por ejemplo:

        {
            "q001": {"d006": 1, "d010": 0},
            "q002": {"d007": 1, "d011": 1}
        }
    """
    
	judgments = {}
	with open(path, encoding = "utf-8") as f:
		for line in f:
			query_id, docs_str = line.strip().split("\t")
			docs = {}
			for pair in docs_str.split(","):
				doc_id, score = pair.split(":")
				docs[doc_id] = int(score)
			judgments[query_id] = docs
	return judgments

In [202]:
"""
Ejemplo de carga de juicios de relevancia.

Se utiliza la función `load_relevance_judgments` para leer un archivo de 
juicios de relevancia en formato TSV. Cada línea del archivo contiene un 
identificador de consulta (`query_id`) y una lista de pares `doc_id:score`, 
que indican la relevancia de los documentos respecto a la consulta.

El resultado es un diccionario donde las claves son los IDs de las consultas
y los valores son diccionarios con los documentos y su puntuación de relevancia.
"""

# CAMBIE esta ruta según la ubicación de los archivos en su sistema
path_judgments = "../data/relevance-judgments.tsv"

judgments = load_relevance_judgments(path_judgments)

print("--------** load relevance judgments **--------\n")
print("Parámetros:\n")
print("- Archivo de juicios de relevancia =", path_judgments)
print("\nResultado (primeras consultas):\n")

# Mostrar solo las primeras 3 consultas
for i, (query_id, docs) in enumerate(judgments.items()):
    print(f"- Consulta {query_id}: {docs}")
    if i >= 2:
        break


--------** load relevance judgments **--------

Parámetros:

- Archivo de juicios de relevancia = ../data/relevance-judgments.tsv

Resultado (primeras consultas):

- Consulta q01: {'d186': 4, 'd254': 5, 'd016': 5}
- Consulta q02: {'d136': 2, 'd139': 2, 'd143': 4, 'd283': 4, 'd228': 4, 'd164': 4, 'd318': 2, 'd291': 4, 'd293': 4, 'd147': 2, 'd149': 2}
- Consulta q03: {'d152': 3, 'd291': 4, 'd283': 4, 'd147': 3, 'd318': 2, 'd105': 2}


In [203]:
def load_results(
        filepath: str
    ) -> dict[str, dict[str, float]]:
    
	"""
    Carga los resultados de recuperación desde un archivo.

    La función lee un archivo donde cada consulta (`query_id`) está listada en
    una línea separada, seguida por una línea que contiene los documentos
    recuperados con sus puntuaciones de similitud o relevancia.  

    El formato esperado del archivo es el siguiente:

        q001
        d259:0.1080,d085:0.0858,d134:0.0501
        q002
        d010:0.2675,d006:0.2110

    Es decir:
    - Una línea con el identificador de consulta (`qXXX`).
    - Una línea con una lista de pares `doc_id:score` separados por comas.

    Parámetros
    ----------
    filepath : str
        Ruta al archivo de resultados. Cada consulta debe estar precedida por
        su `query_id` y seguida de los documentos con sus puntuaciones.

    Retorna
    -------
    dict[str, dict[str, float]]
        Diccionario donde las claves son los `query_id` y los valores son
        diccionarios con los documentos recuperados (`doc_id`) y sus puntuaciones.
        Por ejemplo:

        {
            "q001": {"d259": 0.1080, "d085": 0.0858, "d134": 0.0501},
            "q002": {"d010": 0.2675, "d006": 0.2110}
        }
    """
    
	results = {}
	current_query = None

	with open(filepath, "r", encoding="utf-8") as f:
		for line in f:
			line = line.strip()
			if not line:
				continue
			# Si la línea empieza con qXX entonces es una nueva query
			if line.startswith("q"):
				current_query = line
				results[current_query] = {}
			else:
				pairs = line.split(",")
				for pair in pairs:
					doc_id, score = pair.split(":")
					results[current_query][doc_id] = float(score)
	return results


In [204]:
"""
Ejemplo de carga de resultados de recuperación.

Se utiliza la función `load_results` para leer un archivo con los resultados
de recuperación de consultas. El archivo contiene, para cada consulta, una 
línea con su identificador (`qXXX`) seguida de una línea con los documentos 
recuperados y sus puntuaciones de similitud.

El resultado es un diccionario donde las claves son los IDs de las consultas 
y los valores son diccionarios con los documentos y sus puntuaciones.
"""

# CAMBIE esta ruta según la ubicación de los archivos en su sistema
path_results = "../results/GESIM-consultas_resultados.tsv"

results = load_results(path_results)

print("--------** load results **--------\n")
print("Parámetros:\n")
print("- Archivo de resultados =", path_results)
print("\nResultado (primeras consultas):\n")

# Mostrar solo las 2 primeras consultas 
for i, (query_id, docs) in enumerate(results.items()):
    print(f"- Consulta {query_id}:")
    # Solo cinco documentos por consulta
    for doc_id, score in list(docs.items())[:5]:  
        print(f"   {doc_id}: {score:.4f}")
    if i >= 1:
        break


--------** load results **--------

Parámetros:

- Archivo de resultados = ../results/GESIM-consultas_resultados.tsv

Resultado (primeras consultas):

- Consulta q01:
   d016: 0.1116
   d085: 0.0677
   d259: 0.0654
   d254: 0.0642
   d186: 0.0475
- Consulta q02:
   d147: 0.1198
   d149: 0.0809
   d283: 0.0731
   d291: 0.0483
   d134: 0.0453


In [205]:
def get_relevance_lists(
        query_id: str, 
        results: List[str], 
        judgments: Dict[str, Dict[str, int]]
    ) -> Tuple[List[int], List[int], int]:
    
	"""
	Construye listas de relevancia binaria y de grados de relevancia 
	para una consulta específica.

	Parámetros
	----------
	query_id : str
		Identificador de la consulta.
	results : List[str]
		Lista ordenada de identificadores de documentos recuperados para la consulta.
	judgments : Dict[str, Dict[str, int]]
		Juicios de relevancia en formato:
		{
			query_id: {doc_id: grado_relevancia, ...},
			...
		}

	Retorna
	-------
	Tuple[List[int], List[int], int]
		- relevance_binary : lista binaria (1 si el documento es relevante, 0 en caso contrario).
		- relevance_grades : lista de grados de relevancia (entero ≥ 0).
		- num_relevant : número total de documentos relevantes para la consulta.

	Notas
	-----
	La lista `results` define el orden en que se construyen las listas de relevancia.
	Si un documento no está en los juicios de relevancia, se considera no relevante (0).
	"""

	relevance_binary = []
	relevance_grades = []
	relevant_docs = judgments.get(query_id, {})
	# Se obtiene la lista para cada documento evaluado por el sistema
	for doc_id in results:
		if doc_id in relevant_docs:
			relevance_binary.append(1)
			relevance_grades.append(relevant_docs[doc_id])
		else:
			relevance_binary.append(0)
			relevance_grades.append(0)
	return relevance_binary, relevance_grades, len(relevant_docs)


In [206]:
"""
Ejemplo de construcción de listas de relevancia.

Se utiliza la función `get_relevance_lists` para comparar los resultados 
obtenidos de la consulta `q01` contra los juicios de relevancia cargados.

El proceso retorna tres valores:
1. `relevance_binary`: lista con 1 si el documento recuperado es relevante, 0 en caso contrario.
2. `relevance_grades`: lista con los grados de relevancia (entero ≥ 0).
3. `num_relevant`: número total de documentos relevantes según el juicio.

Esto permite usar la información en la evaluación de medidas como 
Precisión, Recall, MAP o nDCG.
"""

print("--------** get relevance lists **--------\n")
print("Parámetros:\n")
print("- Query = q01\n")

binary, grades, num_relevant = get_relevance_lists("q01", results["q01"].keys(), judgments)

print("Resultado:\n")
print("- Relevancia binaria:", binary)
print("- Grados de relevancia:", grades)
print("- Total relevantes según juicios:", num_relevant)


--------** get relevance lists **--------

Parámetros:

- Query = q01

Resultado:

- Relevancia binaria: [1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
- Grados de relevancia: [5, 0, 0, 5, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
- Total relevantes según juicios: 3


In [207]:
def precision_at_k(
		relevance: Union[List[int], np.ndarray], 
		k: int
	) -> float:

	"""
    Calcula la métrica de Precision@K en recuperación de información.

    Precision@K mide la proporción de documentos relevantes entre los 
    primeros K documentos recuperados. Se utiliza para evaluar la calidad 
    de los resultados en el top-K.

    Fórmula:
        Precision@K = (# documentos relevantes en top-K) / K

    Parámetros
    ----------
    relevance : List[int]
        Lista binaria donde cada posición representa un documento recuperado:
        - 1 indica que el documento es relevante.
        - 0 indica que el documento no es relevante.
    k : int
        Número de documentos del top-K a considerar. 
        Debe ser mayor que 0.

    Retorna
    -------
    float
        Valor de Precision@K en el rango [0.0, 1.0].
        Retorna 0.0 si la lista está vacía.

    Lanza
    -----
    ValueError
        Si el valor de `k` es menor o igual a 0.
    """

	# K debe ser positivo y mayor a cero para poder calcular la métrica
	if k <= 0:
		raise ValueError("k should be greater than 0")
	# Si no se recuperó ningún documento, la métrica es 0.0
	if len(relevance) == 0:
		return 0.0
	relevance = np.array(relevance, dtype = int)
	# Se limita K al número real de documentos recuperados
	k = min(k, len(relevance))
	return float(np.mean(relevance[:k]))

In [208]:
def recall_at_k(
		relevance: Union[List[int], np.ndarray], 
		number_relevant_documents: int, 
		k: int
	) -> float:

	"""
    Calcula la métrica de Recall@K en recuperación de información.

    Recall@K mide la fracción de documentos relevantes recuperados en el top-K,
    en comparación con el número total de documentos relevantes que existen en 
    la colección.

    Fórmula:
        Recall@K = (# documentos relevantes en top-K) / (total documentos relevantes en la colección)

    Parámetros
    ----------
    relevance : List[int]
        Lista binaria donde cada posición representa un documento recuperado:
        - 1 indica que el documento es relevante.
        - 0 indica que el documento no es relevante.
    number_relevant_documents : int
        Número total de documentos relevantes en la colección.
    k : int
        Número de documentos del top-K a considerar.
        Debe ser mayor que 0.

    Retorna
    -------
    float
        Valor de Recall@K en el rango [0.0, 1.0].precision(relevance_query_1)
        Retorna 0.0 si no existen documentos relevantes o si la lista está vacía.

    Lanza
    -----
    ValueError
        Si el valor de `k` es menor o igual a 0.
    """

	# K debe ser positivo y mayor a cero para poder calcular la métrica
	if k <= 0:
		raise ValueError("k should be greater than 0")
	# Si no existen documentos relevantes, la métrica es 0.0
	if number_relevant_documents <= 0:
		return 0.0
	# Si no se recuperó ningún documento, la métrica es 0.0
	if len(relevance) == 0:
		return 0.0
	relevance = np.array(relevance, dtype = int)
	# Se limita K al número real de documentos recuperados
	k = min(k, len(relevance))
	return float(np.sum(relevance[:k]) / number_relevant_documents)

In [209]:
def average_precision(
		relevance: Union[List[int], np.ndarray]
	) -> float:

	"""
    Calcula la métrica de Average Precision (AP) en recuperación de información.

    Average Precision mide la calidad de la recuperación considerando 
    no solo cuántos documentos relevantes aparecen, sino también en qué 
    posiciones se encuentran. 

    La métrica se obtiene calculando la precisión cada vez que se 
    recupera un documento relevante y luego promediando estos valores.

    Fórmula:
        AP = (1 / R) * Σ [ Precision@i * rel(i) ]
    donde:
        - R es el número total de documentos relevantes en la colección.
        - Precision@i es la precisión en la posición i.
        - rel(i) = 1 si el documento en la posición i es relevante, 0 en caso contrario.
    
	Parámetros
    ----------
    relevance : List[int]
        Lista binaria donde cada posición representa un documento recuperado:
        - 1 indica que el documento es relevante.
        - 0 indica que el documento no es relevante.
        Se asume que la lista incluye todos los documentos relevantes de la colección.

    Retorna
    -------
    float
        Valor de Average Precision en el rango [0.0, 1.0].
        Retorna 0.0 si no hay documentos relevantes en la colección.
    """

	relevance = np.array(relevance, dtype = int)
	total_relevant = np.sum(relevance)
	# Si no existen documentos relevantes, la métrica es 0.0
	if total_relevant == 0:
		return 0.0
	# Si no se recuperó ningún documento, la métrica es 0.0
	if len(relevance) == 0:
		return 0.0
	precisions = []
	# Recorrido del vector empezando las posiciones en 1
	for i, rel in enumerate(relevance, start = 1):  
		if rel == 1:
			# Calculo de Precision@i
			precisions.append(precision_at_k(relevance, i))  
	return float(np.sum(precisions) / total_relevant)

In [210]:
def mean_average_precision(
		queries_relevances: List[Union[List[int], np.ndarray]]
	) -> float:
    
	"""
    Calcula la métrica de Mean Average Precision (MAP) en recuperación de información.

    MAP es una extensión de Average Precision (AP) para múltiples consultas.
    Se obtiene calculando el Average Precision de cada consulta y luego
    promediando estos valores.

    Fórmula:	# Si no se recuperaron documentos, el recall@k es 0.0

        MAP = (1 / Q) * Σ [ AP(q) ]
    donde:
        - Q es el número de consultas.
        - AP(q) es el Average Precision de la consulta q.

    Parámetros
    ----------
    queries_relevances : List[List[int]]
        Lista de consultas, donde cada consulta es representada por un vector binario:
        - 1 indica que el documento es relevante.
        - 0 indica que el documento no es relevante.
        Se asume que cada vector incluye todos los documentos relevantes de la colección para esa consulta.

    Retorna
    -------
    float
        Valor de Mean Average Precision en el rango [0.0, 1.0].
        Retorna 0.0 si no se proporciona ninguna consulta.
    """

	# Si no existe ninguna consulta, la métrica es 0.0
	if len(queries_relevances) == 0:
		return 0.0
	# Valores de precision para todas las consultas
	average_precision_values = [average_precision(r) for r in queries_relevances]
	return float(np.mean(average_precision_values))

In [211]:
def dcg_at_k(
		relevance: Union[List[int], np.ndarray], 
		k: int
	) -> float:

	"""
    Calcula la métrica Discounted Cumulative Gain (DCG) hasta la posición K.

    DCG mide la ganancia acumulada de los documentos relevantes, ponderada
    por su posición en la lista de resultados. Documentos relevantes en
    posiciones más altas contribuyen más que aquellos en posiciones más bajas.

    Fórmula:
        DCG@K = Σ ( rel_i / log2(max(i,2)) ), para i = 1..K

    Parámetros
    ----------
    relevance : List[int]
        Lista de relevancias de los documentos ordenados por ranking.
        La relevancia es un número natural (ejemplo: 0, 1, 2, 3, ...).
    k : int
        Posición de corte K hasta la cual se calcula el DCG.

    Retorna
    -------
    float
        Valor de DCG@K (número real >= 0).
        Retorna 0.0 si no hay documentos o si K <= 0.
    """

	# K debe ser positivo y mayor a cero para poder calcular la métrica
	if k <= 0:
		raise ValueError("K should be greater than 0")
	# Si no se recuperó ningún documento, la métrica es 0.0
	if len(relevance) == 0:
		return 0.0
	# Se limita K al número real de documentos recuperados
	k = min(k, len(relevance))
	relevance = np.array(relevance[:k], dtype = float)
	dcg = 0.0
	for i in range(k):
		dcg = dcg + relevance[i] / np.log2(max(i + 1, 2))
	return float(dcg)

In [212]:
def ndcg_at_k(
		relevance: Union[List[int], np.ndarray], 
		k: int
	) -> float:

	"""
    Calcula la métrica Normalized Discounted Cumulative Gain (NDCG) hasta la posición K.

    NDCG es la versión normalizada de DCG, donde se divide el valor de DCG
    entre el DCG ideal (IDCG). De esta manera, NDCG siempre toma valores
    entre 0 y 1, independientemente de la distribución de relevancias.

    Fórmula:
        NDCG@K = DCG@K / IDCG@K

    Parámetros
    ----------
    relevance : List[int]
        Lista de relevancias de los documentos ordenados por ranking.
        La relevancia es un número natural (ejemplo: 0, 1, 2, 3, ...).
    k : int
        Posición de corte K hasta la cual se calcula el NDCG.

    Retorna
    -------
    float
        Valor de NDCG@K (número real entre 0 y 1).
        Retorna 0.0 si no hay documentos, si IDCG = 0 o si K <= 0.
    """
	
	# K debe ser positivo y mayor a cero para poder calcular la métrica
	if k <= 0:
		raise ValueError("K should be greater than 0")
	# Si no se recuperó ningún documento, la métrica es 0.0
	if len(relevance) == 0:
		return 0.0
	# Se calcula el dcg@k
	dcg = dcg_at_k(relevance, k)
	# Se ordena la query para lograr el mayor DCG
	ideal_relevance = sorted(relevance, reverse = True)
	idcg = dcg_at_k(ideal_relevance, k)
	if idcg == 0:
		return 0.0
	return float(dcg / idcg)

In [213]:
def evaluate_query(
        query_id: str, 
        results: list[str], 
        judgments: dict[str, dict[str, int]]
    ) -> dict[str, float]:
    
	"""
    Evalúa el desempeño de una consulta específica usando diferentes métricas
    de recuperación de información.

    Parámetros
    ----------
    query_id : str
        Identificador de la consulta (ej. "q01").
    results : list[str]
        Lista ordenada de identificadores de documentos recuperados por el sistema.
    judgments : dict[str, dict[str, int]]
        Diccionario con los juicios de relevancia. La clave es el ID de la consulta 
        y el valor es un diccionario donde cada clave es un ID de documento y su 
        valor es un entero indicando el grado de relevancia.

    Retorna
    -------
    dict[str, float]
        Diccionario con las métricas calculadas para la consulta:
        - `"P@M"` : Precisión en M (donde M es el número de documentos relevantes).
        - `"R@M"` : Recall en M.
        - `"NDCG@M"` : Normalized Discounted Cumulative Gain en M.
        - `"AP"` : Average Precision (precisión promedio).
    
    Notas
    -----
    Esta función utiliza `get_relevance_lists` para obtener las listas de relevancia 
    y luego aplica funciones auxiliares (`precision_at_k`, `recall_at_k`, `ndcg_at_k`, 
    `average_precision`) para calcular las métricas.
    """
    
	relevance_bin, relevance_grades, num_rel = get_relevance_lists(query_id, results, judgments)

	# Según enunciado se evalúa hasta el número de documentos relevantes
	M = num_rel  
	# Métricas calculadas
	p_at_m = precision_at_k(relevance_bin, M)
	r_at_m = recall_at_k(relevance_bin, num_rel, M)
	ndcg_at_m = ndcg_at_k(relevance_grades, M)
	ap = average_precision(relevance_bin) 
	return {
		"P@M": p_at_m,
		"R@M": r_at_m,
		"NDCG@M": ndcg_at_m,
		"AP": ap
	}

In [214]:
def evaluate_system(
        results: dict[str, list[str]], 
        judgments: dict[str, dict[str, int]]
    ) -> tuple[dict[str, dict[str, float]], float]:
    
	"""
    Evalúa el desempeño de todo el sistema de recuperación sobre un conjunto de consultas.

    Parámetros
    ----------
    results : dict[str, list[str]]
        Diccionario donde la clave es el identificador de la consulta (ej. "q01")
        y el valor es una lista ordenada de identificadores de documentos recuperados.
    judgments : dict[str, dict[str, int]]
        Diccionario de juicios de relevancia. 
        Cada clave es el ID de una consulta y su valor es un diccionario que asigna
        a cada documento un grado de relevancia (entero).

    Retorna
    -------
    tuple[dict[str, dict[str, float]], float]
        Una tupla compuesta por:
        - `per_query` : diccionario con las métricas por consulta, donde cada clave 
          es el ID de la consulta y cada valor es otro diccionario con:
            - `"P@M"` : Precisión en M
            - `"R@M"` : Recall en M
            - `"NDCG@M"` : NDCG en M
            - `"AP"` : Average Precision
        - `MAP` : valor agregado del Mean Average Precision sobre todas las consultas.

    Notas
    -----
    Para cada consulta se invoca `evaluate_query`, y los valores de Average Precision
    se promedian para obtener el MAP (Mean Average Precision).
    """
     
	per_query = {}
	all_ap = []
	# Métricas individuales
	for qid, docs in results.items():
		metrics = evaluate_query(qid, docs, judgments)
		per_query[qid] = metrics
		all_ap.append(metrics["AP"])
	# Métrica general
	MAP = float(np.mean(all_ap)) if all_ap else 0.0
	return per_query, MAP

In [215]:
"""
Ejemplo de evaluación completa del sistema de recuperación.

Se utiliza la función `evaluate_system` para:
1. Comparar los resultados obtenidos (`results`) contra los juicios de relevancia (`judgments`).
2. Calcular métricas por consulta: P@M, R@M, NDCG@M y AP.
3. Obtener el valor global de MAP (Mean Average Precision).

El resultado es una tupla `(per_query, MAP)` donde:
- `per_query` contiene las métricas por consulta.
- `MAP` es un único valor agregado.
"""

evaluation = evaluate_system(results, judgments)

print("--------** evaluate system **--------\n")
print("Métricas por consulta:\n")
for qid, metrics in evaluation[0].items():
    print(f"{qid}: {metrics}")

print("\nMAP global:")
print(evaluation[1])


--------** evaluate system **--------

Métricas por consulta:

q01: {'P@M': 0.3333333333333333, 'R@M': 0.3333333333333333, 'NDCG@M': 0.3992424290497487, 'AP': 0.7000000000000001}
q02: {'P@M': 0.5454545454545454, 'R@M': 0.5454545454545454, 'NDCG@M': 0.5656224204878386, 'AP': 0.6894564571072915}
q03: {'P@M': 1.0, 'R@M': 1.0, 'NDCG@M': 0.9946788263132377, 'AP': 1.0}
q04: {'P@M': 0.7142857142857143, 'R@M': 0.7142857142857143, 'NDCG@M': 0.7752886293631626, 'AP': 0.869047619047619}
q06: {'P@M': 0.6666666666666666, 'R@M': 0.6666666666666666, 'NDCG@M': 0.8053569325627968, 'AP': 0.8551587301587301}
q07: {'P@M': 0.25, 'R@M': 0.25, 'NDCG@M': 0.26064801430716, 'AP': 0.2123015873015873}
q08: {'P@M': 0.6666666666666666, 'R@M': 0.6666666666666666, 'NDCG@M': 0.747192102186424, 'AP': 0.7595790715276008}
q09: {'P@M': 0.8333333333333334, 'R@M': 0.8333333333333334, 'NDCG@M': 0.8878789582207093, 'AP': 0.9333333333333332}
q10: {'P@M': 0.375, 'R@M': 0.375, 'NDCG@M': 0.3043724391664512, 'AP': 0.27965118636371